# Processing

i proccess this into 2 version:
1. Sentiment analysis ready: just cleaning-light without remove stopwords.
2. Advance cleaning -> for topic analysis, clustering, dll.

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('../../processed/concat_data.csv')
df2 = pd.read_csv('../../processed/concat_data.csv')

### slangword normalizing

In [3]:
import json
import pandas as pd

In [4]:
with open('../stopword/slangwords.txt', 'r', encoding='utf-8') as f:
    slang_dict = json.load(f)
    
kamusalay_df = pd.read_csv('../stopword/new_kamusalay.csv', encoding='latin1', header=None, names=['slang', 'baku'])
kamusalay_dict = dict(zip(kamusalay_df['slang'], kamusalay_df['baku']))

def normalize_slang(text):
    words = text.split()
    words = [slang_dict.get(w, w) for w in words]
    words = [kamusalay_dict.get(w, w) for w in words]
    return ' '.join(words)

## Proccess for sentiment

In [5]:
df1.head()

,text
0,Kusus untuk kasus Tom Lembong yang dikasih Abo...
1,"Kalo tom lebong menerima abolisi ini, berarti ..."
2,Menjadi bukti bahwa kebijakan hukum di negara ...
3,Karuan bebasne AE pak..kok pegel2 sidang seng ...
4,"RUWET KAREPE DEWE KAKEAN DRAMA KOYOK JOKOWI, S..."


In [6]:
import re
import emoji

In [7]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    # Ganti koma (dengan/tanpa spasi setelahnya) jadi satu spasi
    text = re.sub(r',\s*', ' ', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\w+|#\w+', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[\u0600-\u06FF]', '', text) #arab
    text = re.sub(r'[^\x00-\x7F]+', '', text) # cina jpn
    text = re.sub(r'[^\w\s]', ' ', text)
    text = emoji.replace_emoji(text, replace=' ')
    text = re.sub(r'(.)\1+', r'\1', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [8]:
# stemming using sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# cara pakai
def stem_text(text):
    return stemmer.stem(text)

In [9]:
df1.isnull().sum()

# drop rows with null values
df1 = df1.dropna(subset=['text'])

In [10]:
def half_clean(text):
    text = clean_text(text)
    text = normalize_slang(text)
    # text = stem_text(text)
    return text

from tqdm import tqdm
tqdm.pandas()
df1['cleaned'] = df1['text'].progress_apply(half_clean)

100%|██████████| 3961/3961 [00:00<00:00, 10353.98it/s]


In [11]:
df1.head()

,text,cleaned
0,Kusus untuk kasus Tom Lembong yang dikasih Abo...,khusus untuk kasus tom lembong yang kasih abol...
1,"Kalo tom lebong menerima abolisi ini, berarti ...",kalau tom lebong menerima abolisi ini berarti ...
2,Menjadi bukti bahwa kebijakan hukum di negara ...,menjadi bukti bahwa kebijakan hukum di negara ...
3,Karuan bebasne AE pak..kok pegel2 sidang seng ...,karuan bebasne saja pak kok pegal sidang seng ...
4,"RUWET KAREPE DEWE KAKEAN DRAMA KOYOK JOKOWI, S...",ruwet karepe sendiri kebanyakan drama koyok jo...


In [12]:
forbidden_words = ['𝗗𝗥𝗔𝗚𝗢𝗡𝟭𝟴𝟳_🀄', 'web', 'jt', 'juta', 'wd', 'hokinya', 'hoki', 'putaranya', 'putaran', 'pecahannya', 'gcr', 'dragon187', 'ngewe', 'farhan', 'gajah', 'vcs', 'tengu', 'carlina', 'Carlina', 'usman', 'wasaf']

# Fungsi cek apakah ada kata terlarang
def contains_forbidden(text):
    if not isinstance(text, str):
        return False
    return any(word in text.lower().split() for word in forbidden_words)

# Hapus baris yang mengandung kata terlarang
df1 = df1[~df1['text'].apply(contains_forbidden)]

In [13]:
df1.isnull().sum()

text       0
cleaned    0
dtype: int64

In [14]:
df1.to_csv('../../processed/sentiment-ready.csv', index=False, encoding='utf-8-sig')

## Advance proccess

In [15]:
def clean_text2(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    # Ganti koma (dengan/tanpa spasi setelahnya) jadi satu spasi
    text = re.sub(r',\s*', ' ', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\w+|#\w+', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[\u0600-\u06FF]', '', text) #arab
    text = re.sub(r'[^\x00-\x7F]+', '', text) # cina jpn
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = re.sub(r'[^\x00-\xFF]+', '', text)
    # Hapus karakter non-alfanumerik kecuali spasi
    text = re.sub(r'[^\w\s]', ' ', text)
    text = emoji.replace_emoji(text, replace=' ')
    # Hilangkan spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    # Hilangkan huruf berulang (2x atau lebih)
    text = re.sub(r'(.)\1{1,}', r'\1', text)
    text = re.sub(r'(.)\1+', r'\1', text)
    return text

In [16]:
df2.head()

,text
0,Kusus untuk kasus Tom Lembong yang dikasih Abo...
1,"Kalo tom lebong menerima abolisi ini, berarti ..."
2,Menjadi bukti bahwa kebijakan hukum di negara ...
3,Karuan bebasne AE pak..kok pegel2 sidang seng ...
4,"RUWET KAREPE DEWE KAKEAN DRAMA KOYOK JOKOWI, S..."


In [17]:
df2.isnull().sum()

text    0
dtype: int64

In [18]:
# drop rows with null values
df2 = df2.dropna(subset=['text'])

### Pakai stopword

In [19]:
with open('../stopword/stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = set([line.strip() for line in f if line.strip()])

In [20]:
def remove_stopwords(text):
    if not isinstance(text, str):
        return ""
    words = text.split()
    filtered = [w for w in words if w not in stopwords]
    return ' '.join(filtered)

In [21]:
# stopword removal using Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
stop_factory = StopWordRemoverFactory()
stopword = stop_factory.create_stop_word_remover()

def remove_stopwords2(text):
    return stopword.remove(text)

In [22]:
def full_clean(text):
    text = clean_text2(text)
    text = normalize_slang(text)
    text = stem_text(text)
    text = remove_stopwords(text)
    text = remove_stopwords2(text)
    return text

from tqdm import tqdm
tqdm.pandas()
df2['cleaned'] = df2['text'].progress_apply(full_clean)

100%|██████████| 3961/3961 [04:28<00:00, 14.75it/s]


In [23]:
df2.head()

,text,cleaned
0,Kusus untuk kasus Tom Lembong yang dikasih Abo...,khusus kasus tom lembong kasih abolisi preside...
1,"Kalo tom lebong menerima abolisi ini, berarti ...",tom lebong terima abolisi arti terima diangap ...
2,Menjadi bukti bahwa kebijakan hukum di negara ...,jadi bukti bijak hukum negara formalitas publi...
3,Karuan bebasne AE pak..kok pegel2 sidang seng ...,karuan bebasne pak pegal sidang seng demo seng...
4,"RUWET KAREPE DEWE KAKEAN DRAMA KOYOK JOKOWI, S...",ruwet karepe drama koyok jokowi sistem birokra...


In [24]:
df2 = df2[~df2['text'].apply(contains_forbidden)]

In [27]:

# Replace all occurrences of 'wkwk' with an empty string in both 'text' and 'cleaned' columns
# df2['text'] = df2['text'].str.replace('wkwk', '', regex=False)
df2['cleaned'] = df2['cleaned'].str.replace('wkwk', '', regex=False)

In [28]:
df2.isnull().sum()

text       0
cleaned    0
dtype: int64

In [29]:
df2.to_csv('../../processed/advanced-cleaning.csv', index=False, encoding='utf-8-sig')